## Import Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split , RandomizedSearchCV , GridSearchCV
from sklearn.svm import SVC , LinearSVC
from sklearn.metrics import classification_report
import m2cgen
import os
import matplotlib.pyplot as plt
import joblib
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width
import sys
import threading

Activity Mapper:

0 : walking
1 : jogging
2 : upstairs
3 : downstairs
4 : sitting
5 : standing
6 : sleeping

## Load Data

In [3]:
dataset_path = os.getcwd().replace('/notebook' , '') + "/dataset/"

# Read training data
imu_df1 = pd.read_csv(dataset_path + "imu_data.csv")
imu_df2 = pd.read_csv(dataset_path + "imu_data_v2.csv")
imu_df1 = imu_df1.iloc[:,1:]
imu_df2 = imu_df2.iloc[:,1:]
imu_df2 = imu_df2.drop(columns = (imu_df2.iloc[:,12:16]).columns , axis= 1)
imu_df1.columns = imu_df1.columns.str.replace("'" , "")
imu_df2.columns = imu_df2.columns.str.replace("'" , "")

imu_df = pd.concat([imu_df1, imu_df2] , axis= 0)

In [4]:
imu_df.columns = imu_df.columns.str.replace("'" , "")
x = imu_df.iloc[:,:-1]
y = imu_df.iloc[:,-1]

In [5]:
x.columns

Index(['attitude.roll', ' attitude.pitch', ' attitude.yaw', ' gravity.x', ' gravity.y', ' gravity.z', ' rotationRate.x', ' rotationRate.y', ' rotationRate.z', ' userAcceleration.x', ' userAcceleration.y', ' userAcceleration.z'], dtype='object')

# Modeling

## Movement Classification

In [6]:
x1, x_test , y1 , y_test = train_test_split(x ,y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [7]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge'],
    'multi_class': ['ovr' , 'crammer_singer']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

KeyboardInterrupt: 

In [25]:
svc = LinearSVC(C = 100, loss= 'hinge' , penalty= 'l1' , multi_class= 'crammer_singer')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.68      0.73      0.70      1768
           1       0.62      0.72      0.66      1646
           2       0.79      0.70      0.74      2158
           3       0.91      0.90      0.91      1948
           4       0.66      0.74      0.70      1985
           5       0.79      0.70      0.74      2503
           6       0.94      0.88      0.91      2392

    accuracy                           0.77     14400
   macro avg       0.77      0.77      0.77     14400
weighted avg       0.78      0.77      0.77     14400



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [19]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      7815
           1       0.91      0.96      0.93      7298
           2       0.94      0.91      0.93      7924
           3       0.97      0.97      0.97      7685
           4       0.98      0.98      0.98      9012
           5       0.98      0.98      0.98      8906
           6       1.00      1.00      1.00      8960

    accuracy                           0.96     57600
   macro avg       0.96      0.96      0.96     57600
weighted avg       0.96      0.96      0.96     57600



In [20]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      2426
           1       0.92      0.95      0.94      2316
           2       0.93      0.91      0.92      2474
           3       0.97      0.97      0.97      2385
           4       0.98      0.98      0.98      2809
           5       0.97      0.98      0.98      2791
           6       1.00      1.00      1.00      2799

    accuracy                           0.96     18000
   macro avg       0.96      0.96      0.96     18000
weighted avg       0.96      0.96      0.96     18000



In [21]:
with open(os.getcwd().replace('/notebook' , '') + "/model/activity_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/activity_model.py" ))

RecursionError: maximum recursion depth exceeded

## Motion Classification

In [6]:
"""
Map
0 - steady motion,
1 - unsteady motion
"""
mapper = {
    0 : 1,
    1 : 1,
    2 : 1,
    3 : 1,
    4 : 0,
    5 : 0,
    6 : 0
}

In [7]:
motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x ,motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [10]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge'],
    'multi_class': ['ovr' , 'crammer_singer']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
19,0.707625,"{'C': 10, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l2'}"
22,0.706251,"{'C': 10, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"
23,0.705511,"{'C': 10, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l2'}"
18,0.705297,"{'C': 10, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"
13,0.705238,"{'C': 1, 'loss': 'hinge', 'multi_class': 'ovr', 'penalty': 'l2'}"
10,0.704566,"{'C': 1, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"
15,0.704342,"{'C': 1, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l2'}"
11,0.704079,"{'C': 1, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l2'}"
2,0.703982,"{'C': 0.1, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"
6,0.703972,"{'C': 0.1, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"


In [8]:
svc = LinearSVC(C = 10, loss= 'squared_hinge' , multi_class= 'crammer_singer', penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.69      0.65      0.67     11796
           1       0.72      0.76      0.74     13867

    accuracy                           0.71     25663
   macro avg       0.70      0.70      0.70     25663
weighted avg       0.70      0.71      0.70     25663



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [9]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           0       0.69      0.65      0.67     46631
           1       0.73      0.76      0.74     56018

    accuracy                           0.71    102649
   macro avg       0.71      0.71      0.71    102649
weighted avg       0.71      0.71      0.71    102649



In [10]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           0       0.69      0.66      0.67     14422
           1       0.73      0.76      0.74     17656

    accuracy                           0.71     32078
   macro avg       0.71      0.71      0.71     32078
weighted avg       0.71      0.71      0.71     32078



In [11]:
with open(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" ))

446


## Steady Movement Classification

In [12]:
steady_df  =imu_df[(imu_df['Target'] == 4) | (imu_df['Target'] == 5) | (imu_df['Target'] == 6)]
x = steady_df.iloc[:,:-1]
y = steady_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [12]:
svc = LinearSVC()

param_dist = param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge'],
    'multi_class': ['ovr' , 'crammer_singer']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
9,0.573517,"{'C': 1, 'loss': 'squared_hinge', 'multi_class': 'ovr', 'penalty': 'l2'}"
31,0.570430,"{'C': 100, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l2'}"
30,0.568568,"{'C': 100, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"
26,0.568205,"{'C': 100, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"
18,0.567887,"{'C': 10, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"
22,0.567728,"{'C': 10, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"
19,0.567728,"{'C': 10, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l2'}"
23,0.567456,"{'C': 10, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l2'}"
27,0.567410,"{'C': 100, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer', 'penalty': 'l2'}"
14,0.565640,"{'C': 1, 'loss': 'hinge', 'multi_class': 'crammer_singer', 'penalty': 'l1'}"


In [13]:
svc = LinearSVC(C = 1, loss= 'squared_hinge' , multi_class= 'ovr', penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           4       0.27      0.57      0.36      1725
           5       0.60      0.57      0.58      3856
           6       0.86      0.58      0.69      5432

    accuracy                           0.58     11013
   macro avg       0.58      0.57      0.55     11013
weighted avg       0.68      0.58      0.60     11013



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [14]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           4       0.28      0.58      0.38      7030
           5       0.60      0.57      0.59     15501
           6       0.86      0.58      0.70     21520

    accuracy                           0.58     44051
   macro avg       0.58      0.58      0.55     44051
weighted avg       0.68      0.58      0.61     44051



In [15]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           4       0.28      0.58      0.38      2250
           5       0.61      0.58      0.59      4874
           6       0.86      0.59      0.70      6642

    accuracy                           0.58     13766
   macro avg       0.58      0.58      0.56     13766
weighted avg       0.68      0.58      0.61     13766



In [16]:
with open(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" ))

1287


## Unsteady Movement Classification

In [17]:
unsteady_df  =imu_df[(imu_df['Target'] != 4) & (imu_df['Target'] != 5) & (imu_df['Target'] != 6)]
x = unsteady_df.iloc[:,:-1]
y = unsteady_df.iloc[:,-1]

## Unsteady Motion Classification

In [18]:
"""
Map
0 - surface motion,
1 - staircase motion
"""
mapper = {
    0 : 0,
    1 : 0,
    2 : 1,
    3 : 1
}

In [19]:
unsteady_motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x ,unsteady_motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [16]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
7,0.611847,"{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}"
3,0.567511,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}"
5,0.567408,"{'C': 1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
1,0.553722,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
9,0.547869,"{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}"
13,0.539318,"{'C': 100, 'loss': 'squared_hinge', 'penalty': 'l2'}"
11,0.538107,"{'C': 10, 'loss': 'hinge', 'penalty': 'l2'}"
15,0.491143,"{'C': 100, 'loss': 'hinge', 'penalty': 'l2'}"
0,NaN,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l1'}"
2,NaN,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l1'}"


In [20]:
svc = LinearSVC(C = 1, loss= 'hinge' , multi_class= 'crammer_singer', penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.58      0.68      0.62      6901
           1       0.66      0.57      0.61      7749

    accuracy                           0.62     14650
   macro avg       0.62      0.62      0.62     14650
weighted avg       0.62      0.62      0.62     14650



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [21]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           0       0.58      0.68      0.63     27317
           1       0.68      0.57      0.62     31281

    accuracy                           0.62     58598
   macro avg       0.63      0.63      0.62     58598
weighted avg       0.63      0.62      0.62     58598



In [22]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           0       0.59      0.69      0.63      8571
           1       0.68      0.57      0.62      9741

    accuracy                           0.63     18312
   macro avg       0.63      0.63      0.63     18312
weighted avg       0.63      0.63      0.63     18312



In [23]:
with open(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" ))

450


## Staircase motion

In [24]:
staircase_df  =imu_df[(imu_df['Target'] == 1) | (imu_df['Target'] == 2)]
x = staircase_df.iloc[:,:-1]
y = staircase_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [18]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
    }


random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
7,0.677375,"{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}"
3,0.674025,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}"
5,0.636998,"{'C': 1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
11,0.623459,"{'C': 10, 'loss': 'hinge', 'penalty': 'l2'}"
1,0.590487,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
9,0.586888,"{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}"
15,0.582304,"{'C': 100, 'loss': 'hinge', 'penalty': 'l2'}"
13,0.542846,"{'C': 100, 'loss': 'squared_hinge', 'penalty': 'l2'}"
0,NaN,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l1'}"
2,NaN,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l1'}"


In [25]:
svc = LinearSVC(C = 1, loss= 'squared_hinge' , multi_class= 'crammer_singer', penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.58      0.76      0.66      2852
           2       0.79      0.62      0.70      4238

    accuracy                           0.68      7090
   macro avg       0.69      0.69      0.68      7090
weighted avg       0.71      0.68      0.68      7090



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [26]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           1       0.58      0.76      0.66     11540
           2       0.79      0.63      0.70     16818

    accuracy                           0.68     28358
   macro avg       0.69      0.70      0.68     28358
weighted avg       0.71      0.68      0.69     28358



In [27]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           1       0.58      0.75      0.66      3631
           2       0.79      0.63      0.70      5231

    accuracy                           0.68      8862
   macro avg       0.68      0.69      0.68      8862
weighted avg       0.70      0.68      0.68      8862



In [28]:
with open(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" ))

445


## Surface motion classification

In [29]:
surface_df  =imu_df[(imu_df['Target'] == 0) | (imu_df['Target'] == 4)]
x = surface_df.iloc[:,:-1]
y = surface_df.iloc[:,-1]

x1, x_test , y1 , y_test = train_test_split(x , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [20]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
7,0.755345,"{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}"
3,0.753836,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}"
11,0.733494,"{'C': 10, 'loss': 'hinge', 'penalty': 'l2'}"
1,0.660714,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
5,0.658922,"{'C': 1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
9,0.646564,"{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}"
15,0.620503,"{'C': 100, 'loss': 'hinge', 'penalty': 'l2'}"
13,0.617767,"{'C': 100, 'loss': 'squared_hinge', 'penalty': 'l2'}"
0,NaN,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l1'}"
2,NaN,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l1'}"


In [30]:
svc = LinearSVC(C = 1, loss= 'hinge' , multi_class= 'crammer_singer', penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.64      0.87      0.74      3151
           4       0.89      0.68      0.77      4801

    accuracy                           0.76      7952
   macro avg       0.77      0.78      0.76      7952
weighted avg       0.79      0.76      0.76      7952



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [31]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           0       0.64      0.87      0.74     12683
           4       0.88      0.68      0.77     19125

    accuracy                           0.75     31808
   macro avg       0.76      0.77      0.75     31808
weighted avg       0.79      0.75      0.76     31808



In [32]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           0       0.64      0.87      0.74      3949
           4       0.89      0.68      0.77      5991

    accuracy                           0.76      9940
   macro avg       0.77      0.78      0.76      9940
weighted avg       0.79      0.76      0.76      9940



In [33]:
with open(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" ))

442
